<a href="https://colab.research.google.com/github/rediahmds/eco-sort/blob/main/train/train_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prepare dataset

In [1]:
!pip install kagglehub

### Download

In [2]:
import kagglehub

alistair_ds = kagglehub.dataset_download("alistairking/recyclable-and-household-waste-classification")
print("Path to dataset files:", alistair_ds)

mostafa_ds = kagglehub.dataset_download("mostafaabla/garbage-classification")
print("Path to dataset files:", mostafa_ds)

joe_ds = kagglehub.dataset_download("joebeachcapital/realwaste")
print("Path to dataset files:", joe_ds)

glhdamar_ds = kagglehub.dataset_download("glhdamar/new-trash-classfication-dataset")
print("Path to dataset files:", glhdamar_ds)

Path to dataset files: /kaggle/input/recyclable-and-household-waste-classification
Path to dataset files: /kaggle/input/garbage-classification
Path to dataset files: /kaggle/input/realwaste
Path to dataset files: /kaggle/input/new-trash-classfication-dataset


Show directory tree

In [3]:
from pathlib import Path

def print_directory_tree(root: Path, prefix: str = ""):
    """
    Mencetak struktur direktori dengan tampilan seperti pohon.
    Hanya menampilkan folder (tanpa file).
    """
    subdirs = sorted([p for p in root.iterdir() if p.is_dir()])
    for i, subdir in enumerate(subdirs):
        connector = "└── " if i == len(subdirs) - 1 else "├── "
        print(f"{prefix}{connector}{subdir.name}")
        extension = "    " if i == len(subdirs) - 1 else "│   "
        print_directory_tree(subdir, prefix + extension)

# Path ke folder utama
alistair_path = Path(alistair_ds) / "images" / "images"
mostafa_path = Path(mostafa_ds) / "garbage_classification"
joe_path = Path(joe_ds) / "realwaste-main" / "RealWaste"
glhdamar_path = Path(glhdamar_ds) / "new-dataset-trash-type-v2"

# Cetak pohon direktori
print(alistair_path.name)
print_directory_tree(alistair_path)

print(mostafa_path.name)
print_directory_tree(mostafa_path)

print(joe_path.name)
print_directory_tree(joe_path)

print(glhdamar_ds)
print_directory_tree(glhdamar_path)


images
├── aerosol_cans
│   ├── default
│   └── real_world
├── aluminum_food_cans
│   ├── default
│   └── real_world
├── aluminum_soda_cans
│   ├── default
│   └── real_world
├── cardboard_boxes
│   ├── default
│   └── real_world
├── cardboard_packaging
│   ├── default
│   └── real_world
├── clothing
│   ├── default
│   └── real_world
├── coffee_grounds
│   ├── default
│   └── real_world
├── disposable_plastic_cutlery
│   ├── default
│   └── real_world
├── eggshells
│   ├── default
│   └── real_world
├── food_waste
│   ├── default
│   └── real_world
├── glass_beverage_bottles
│   ├── default
│   └── real_world
├── glass_cosmetic_containers
│   ├── default
│   └── real_world
├── glass_food_jars
│   ├── default
│   └── real_world
├── magazines
│   ├── default
│   └── real_world
├── newspaper
│   ├── default
│   └── real_world
├── office_paper
│   ├── default
│   └── real_world
├── paper_cups
│   ├── default
│   └── real_world
├── plastic_cup_lids
│   ├── default
│   └── real_world
├── pl

### Copy Dataset

In [4]:
from pathlib import Path
import shutil
import random

def copy_n_files(src_dir, dst_dir, n, randomize=False):
    src_path = Path(src_dir)
    dst_path = Path(dst_dir)

    # Buat folder tujuan jika belum ada
    dst_path.mkdir(parents=True, exist_ok=True)

    # Ambil semua file dari direktori sumber
    all_files = [f for f in src_path.iterdir() if f.is_file()]

    # Pastikan n tidak lebih besar dari jumlah file
    n = min(n, len(all_files))

    # Tentukan file mana yang akan disalin
    if randomize:
        files_to_copy = random.sample(all_files, n)
    else:
        files_to_copy = sorted(all_files)[:n]

    # Copy file satu per satu
    for file in files_to_copy:
        shutil.copy(file, dst_path)
        print(f"Copied: {file.name}")

    print(f"\nTotal {n} files copied from '{src_dir}' to '{dst_dir}' (random: {randomize}).")


#### Customize Alistair Dataset

In [5]:
from pathlib import Path
import shutil
from tqdm import tqdm

source_root = alistair_path
target_root = Path("dataset/train")
target_root.mkdir(parents=True, exist_ok=True)

class_map = {
    "food_waste": "organic",
    "eggshells": "organic",
    "coffee_grounds": "organic",
    "tea_bags": "organic",
    "plastic_soda_bottles": "plastic",
    "plastic_trash_bags": "plastic",
    "plastic_food_containers": "plastic",
    "plastic_shopping_bags": "plastic",
    "plastic_straws": "plastic",
    "plastic_water_bottles": "plastic",
    "plastic_detergent_bottles": "plastic",
    "plastic_cup_lids": "plastic",
    "glass_food_jars": "glass",
    "glass_beverage_bottles": "glass",
    "glass_cosmetic_containers": "glass",
    "aluminum_soda_cans": "metal",
    "aluminum_food_cans": "metal",
    "steel_food_cans": "metal",
    "aerosol_cans": "metal",
    "cardboard_boxes": "paper",
    "cardboard_packaging": "paper",
    "magazines": "paper",
    "newspaper": "paper",
    "office_paper": "paper",
    "paper_cups": "paper",
    "styrofoam_cups": "styrofoam",
    "styrofoam_food_containers": "styrofoam",
    "clothing": "textiles",
    "shoes": "textiles"
}

print("🚀 Memulai pengelompokan dataset dengan penamaan ulang...\n")

for class_name, parent_class in class_map.items():
    for subset in ["default", "real_world"]:
        class_dir = source_root / class_name / subset
        if class_dir.exists():
            img_list = list(class_dir.glob("*.*"))
            print(f"📁 Menyalin {len(img_list)} gambar dari '{class_name}/{subset}' ke '{parent_class}'")
            for i, img in enumerate(tqdm(img_list, desc=f"{class_name}/{subset}", leave=False)):
                dest_dir = target_root / parent_class
                dest_dir.mkdir(parents=True, exist_ok=True)

                # Format nama: subset_class_####__asli.ext
                ext = img.suffix
                original_name = img.stem.replace(" ", "_")
                new_name = f"{subset}_{class_name}_{i:04d}__{original_name}{ext}"
                shutil.copy(img, dest_dir / new_name)

print("\n✅ Pengelompokan selesai tanpa konflik penamaan.")
print("📂 Dataset tersimpan di:", target_root.resolve())


🚀 Memulai pengelompokan dataset dengan penamaan ulang...

📁 Menyalin 250 gambar dari 'food_waste/default' ke 'organic'


📁 Menyalin 250 gambar dari 'food_waste/real_world' ke 'organic'


📁 Menyalin 250 gambar dari 'eggshells/default' ke 'organic'


📁 Menyalin 250 gambar dari 'eggshells/real_world' ke 'organic'


📁 Menyalin 250 gambar dari 'coffee_grounds/default' ke 'organic'


...

📁 Menyalin 250 gambar dari 'clothing/default' ke 'textiles'


📁 Menyalin 250 gambar dari 'clothing/real_world' ke 'textiles'


📁 Menyalin 250 gambar dari 'shoes/default' ke 'textiles'


📁 Menyalin 250 gambar dari 'shoes/real_world' ke 'textiles'



✅ Pengelompokan selesai tanpa konflik penamaan.
📂 Dataset tersimpan di: /content/dataset/train


In [6]:
# Uncomment all for first run

copy_n_files(f"{mostafa_ds}/garbage_classification/paper", "dataset/train/paper", 500, randomize=True)
copy_n_files(f"{mostafa_ds}/garbage_classification/cardboard", "dataset/train/paper", 500, randomize=True)

copy_n_files(f"{mostafa_ds}/garbage_classification/white-glass", "dataset/train/glass", 600, randomize=True)
copy_n_files(f"{mostafa_ds}/garbage_classification/brown-glass", "dataset/train/glass", 600, randomize=True)
copy_n_files(f"{mostafa_ds}/garbage_classification/green-glass", "dataset/train/glass", 600, randomize=True)

copy_n_files(f"{mostafa_ds}/garbage_classification/clothes", "dataset/train/textiles", 1500, randomize=True)
copy_n_files(f"{mostafa_ds}/garbage_classification/shoes", "dataset/train/textiles", 1500, randomize=True)

copy_n_files(f"{mostafa_ds}/garbage_classification/metal", "dataset/train/metal", 750, randomize=True)
copy_n_files(f"{joe_ds}/realwaste-main/RealWaste/Metal", "dataset/train/metal", 750, randomize=True)

copy_n_files(f"{mostafa_ds}/garbage_classification/biological", "dataset/train/organic", 980, randomize=True)
copy_n_files(f"{glhdamar_ds}/new-dataset-trash-type-v2/organic", "dataset/train/organic", 960, randomize=True)



Streaming output truncated to the last 5000 lines.
Copied: clothes1899.jpg
Copied: clothes4564.jpg
Copied: clothes3462.jpg
Copied: clothes2186.jpg
Copied: clothes4272.jpg
Copied: clothes1572.jpg
Copied: clothes1215.jpg
Copied: clothes3683.jpg
Copied: clothes426.jpg
Copied: clothes1792.jpg
Copied: clothes4984.jpg
Copied: clothes1704.jpg
Copied: clothes4970.jpg
Copied: clothes894.jpg
Copied: clothes4731.jpg
Copied: clothes3021.jpg
Copied: clothes597.jpg
Copied: clothes3259.jpg
Copied: clothes2870.jpg
Copied: clothes1564.jpg
Copied: clothes150.jpg
Copied: clothes1993.jpg
Copied: clothes4052.jpg
Copied: clothes3812.jpg
Copied: clothes4986.jpg
Copied: clothes34.jpg
Copied: clothes1989.jpg
Copied: clothes1598.jpg
Copied: clothes1798.jpg
Copied: clothes3175.jpg
Copied: clothes837.jpg
Copied: clothes2966.jpg
Copied: clothes4439.jpg
Copied: clothes670.jpg
Copied: clothes2113.jpg
Copied: clothes1061.jpg
Copied: clothes2209.jpg
Copied: clothes2230.jpg
Copied: clothes4116.jpg
Copied: clothes4662.j

#### TODO: Check for duplicates

In [7]:
!pip install imagehash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 5.3 MB/s eta 0:00:00


In [16]:
from pathlib import Path
from PIL import Image
import imagehash
import torch
import torch.nn.functional as F
from torchvision import models, transforms
from collections import defaultdict
import matplotlib.pyplot as plt

def get_resnet_embedding(img: Image.Image, model, transform, device):
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model(img_tensor).squeeze()
    return features.cpu()

def find_duplicate_images(
    dataset_dir: Path | str,
    hash_threshold: int = 5,
    sim_threshold: float = 0.98,
    delete: bool = False,
    show_preview: bool = True
):
    dataset_dir = Path(dataset_dir)
    print(f"🔍 Mendeteksi duplikat di dalam: {dataset_dir.resolve()}")

    # === Inisialisasi hash & CNN
    hash_dict = defaultdict(list)
    duplicates = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cnn_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    cnn_model = torch.nn.Sequential(*list(cnn_model.children())[:-1])
    cnn_model.eval().to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    counter = 0

    for img_path in dataset_dir.rglob("*.*"):
        if img_path.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
            continue
        try:
            img = Image.open(img_path).convert("RGB")
            h = imagehash.average_hash(img)

            # 🔁 Bandingkan dengan semua hash yang mirip
            for existing_hash in hash_dict:
                if h - existing_hash <= hash_threshold:
                    for existing_path in hash_dict[existing_hash]:
                        try:
                            img1 = Image.open(existing_path).convert("RGB")
                            emb1 = get_resnet_embedding(img1, cnn_model, transform, device)
                            emb2 = get_resnet_embedding(img, cnn_model, transform, device)

                            sim = F.cosine_similarity(emb1, emb2, dim=0).item()
                            if sim >= sim_threshold:
                                counter += 1
                                print(f"🔢 Kode duplikat: {counter}")
                                print(f"⚠️ Duplikat terdeteksi:")
                                print(f"  Original : {existing_path.name}")
                                print(f"  Duplicate: {img_path.name}")
                                print(f"  🔗 Cosine Similarity: {sim:.4f}")

                                duplicates.append(img_path)

                                if show_preview:
                                    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
                                    ax[0].imshow(img1)
                                    ax[0].set_title(f"Original: {existing_path.name}")
                                    ax[1].imshow(img)
                                    ax[1].set_title(f"Duplicate: {img_path.name}")
                                    for a in ax:
                                        a.axis("off")
                                    plt.tight_layout()
                                    plt.show()

                                print("\n")

                                break  # cukup validasi 1 yang cocok

                        except Exception as e:
                            print(f"❌ Error validasi CNN: {e}")
                    else:
                        continue
                    break  # keluar dari hash loop jika sudah match

            hash_dict[h].append(img_path)

        except Exception as e:
            print(f"❌ Gagal membuka {img_path}: {e}")

    # === Hapus jika diminta
    if delete:
        for dup in duplicates:
            try:
                dup.unlink()
                print(f"🗑️ Menghapus: {dup}")
            except Exception as e:
                print(f"❌ Gagal menghapus {dup}: {e}")

    print(f"\n✅ Total duplikat terverifikasi: {len(duplicates)}")
    return duplicates


def find_duplicate_images_(
    dataset_dir: Path | str,
    hash_threshold: int = 5,
    sim_threshold: float = 0.98,
    delete: bool = False,
    show_preview: bool = True
):
    dataset_dir = Path(dataset_dir)
    print(f"🔍 Mendeteksi duplikat di dalam: {dataset_dir.resolve()}")

    hash_dict = defaultdict(list)
    duplicates = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cnn_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    cnn_model = torch.nn.Sequential(*list(cnn_model.children())[:-1])
    cnn_model.eval().to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    counter = 0

    for img_path in dataset_dir.rglob("*.*"):
        if img_path.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
            continue
        try:
            img = Image.open(img_path).convert("RGB")
            h = imagehash.average_hash(img)

            for existing_hash in hash_dict:
                if h - existing_hash <= hash_threshold:
                    for existing_path in hash_dict[existing_hash]:
                        try:
                            img1 = Image.open(existing_path).convert("RGB")
                            emb1 = get_resnet_embedding(img1, cnn_model, transform, device)
                            emb2 = get_resnet_embedding(img, cnn_model, transform, device)

                            sim = F.cosine_similarity(emb1, emb2, dim=0).item()
                            if sim >= sim_threshold:
                                counter += 1
                                print(f"🔢 Kode duplikat: {counter}")
                                print(f"⚠️ Duplikat terdeteksi:")
                                print(f"  Original : {existing_path.name}")
                                print(f"  Duplicate: {img_path.name}")
                                print(f"  🔗 Cosine Similarity: {sim:.4f}")

                                if show_preview:
                                    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
                                    ax[0].imshow(img1)
                                    ax[0].set_title(f"Original: {existing_path.name}")
                                    ax[1].imshow(img)
                                    ax[1].set_title(f"Duplicate: {img_path.name}")
                                    for a in ax:
                                        a.axis("off")
                                    plt.tight_layout()
                                    plt.show()

                                if delete:
                                    try:
                                        img_path.unlink()
                                        print(f"🗑️ Duplikat dihapus: {img_path}")
                                    except Exception as e:
                                        print(f"❌ Gagal menghapus {img_path}: {e}")
                                else:
                                    duplicates.append(img_path)

                                print("\n")
                                break  # match pertama cukup
                        except Exception as e:
                            print(f"❌ Error validasi CNN: {e}")
                    else:
                        continue
                    break  # hash cocok, stop

            hash_dict[h].append(img_path)

        except Exception as e:
            print(f"❌ Gagal membuka {img_path}: {e}")

    print(f"\n✅ Total duplikat terverifikasi: {counter}")
    return duplicates

def scan_duplicate_images(
    dataset_dir: Path | str,
    hash_threshold: int = 5,
    sim_threshold: float = 0.98,
    immediate_delete: bool = False,
    show_preview: bool = True
):
    dataset_dir = Path(dataset_dir)
    print(f"🔍 Mendeteksi duplikat di dalam: {dataset_dir.resolve()}")

    hash_dict = defaultdict(list)
    duplicates = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cnn_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    cnn_model = torch.nn.Sequential(*list(cnn_model.children())[:-1])
    cnn_model.eval().to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    counter = 0

    for img_path in dataset_dir.rglob("*.*"):
        if img_path.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
            continue
        try:
            img = Image.open(img_path).convert("RGB")
            h = imagehash.average_hash(img)

            for existing_hash in hash_dict:
                if h - existing_hash <= hash_threshold:
                    for existing_path in hash_dict[existing_hash]:
                        try:
                            img1 = Image.open(existing_path).convert("RGB")
                            emb1 = get_resnet_embedding(img1, cnn_model, transform, device)
                            emb2 = get_resnet_embedding(img, cnn_model, transform, device)

                            sim = F.cosine_similarity(emb1, emb2, dim=0).item()
                            if sim >= sim_threshold:
                                counter += 1
                                print(f"🔢 Kode duplikat: {counter}")
                                print(f"⚠️ Duplikat terdeteksi:")
                                print(f"  Original : {existing_path.name}")
                                print(f"  Duplicate: {img_path.name}")
                                print(f"  🔗 Cosine Similarity: {sim:.4f}")

                                if show_preview:
                                    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
                                    ax[0].imshow(img1)
                                    ax[0].set_title(f"Original: {existing_path.name}")
                                    ax[1].imshow(img)
                                    ax[1].set_title(f"Duplicate: {img_path.name}")
                                    for a in ax:
                                        a.axis("off")
                                    plt.tight_layout()
                                    plt.show()

                                if immediate_delete:
                                    try:
                                        img_path.unlink()
                                        print(f"🗑️ Duplikat dihapus: {img_path}")
                                    except Exception as e:
                                        print(f"❌ Gagal menghapus {img_path}: {e}")
                                else:
                                    duplicates.append(img_path)

                                print("\n")
                                break  # match pertama cukup
                        except Exception as e:
                            print(f"❌ Error validasi CNN: {e}")
                    else:
                        continue
                    break  # hash cocok, stop

            hash_dict[h].append(img_path)

        except Exception as e:
            print(f"❌ Gagal membuka {img_path}: {e}")

    print(f"\n✅ Total duplikat terverifikasi: {counter}")
    return duplicates



In [ ]:
scan_duplicate_images(
    "dataset/train",
    hash_threshold=6,
    immediate_delete=True,
    show_preview=False
)

### Create Validation Dataset

This dataset will be created by moving some files from training dataset.

In [ ]:
def move_validation_split_custom(train_dir, val_dir, per_class_counts: dict, random_select=True):
    train_dir = Path(train_dir)
    val_dir = Path(val_dir)
    val_dir.mkdir(parents=True, exist_ok=True)

    for class_name, n in per_class_counts.items():
        class_dir = train_dir / class_name
        if not class_dir.exists():
            print(f"⚠️ Folder tidak ditemukan: {class_dir}")
            continue

        images = sorted([p for p in class_dir.glob("*.*") if p.suffix.lower() in {'.jpg', '.jpeg', '.png'}])
        selected = random.sample(images, min(n, len(images))) if random_select else images[:n]
        val_class_dir = val_dir / class_name
        val_class_dir.mkdir(parents=True, exist_ok=True)

        print(f"📁 {class_name}: Memindahkan {len(selected)} file...")
        for img in tqdm(selected, desc=f"  Pindah {class_name}", leave=False):
            shutil.move(str(img), str(val_class_dir / img.name))

    print("\n✅ Selesai membuat validasi set proporsional.")


In [ ]:
# 15% validasi per kelas
per_class_counts = {
    "glass": 495,
    "metal": 525,
    "organic": 300,
    "paper": 600,
    "plastic": 600,
    "styrofoam": 150,
    "textiles": 600
}

move_validation_split_custom("dataset/train", "dataset/test", per_class_counts, random_select=True)

### Data Distribution checking

In [ ]:
from collections import Counter
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder("dataset/train")
label_counts = Counter([label for _, label in train_dataset])
print("Label mapping:", train_dataset.class_to_idx)
print("Distribusi kelas:", label_counts)

test_dataset = ImageFolder("dataset/test")
label_counts = Counter([label for _, label in test_dataset])
print("Distribusi kelas:", label_counts)


### Connect to Google Drive (Optional)

This functionality allows to save the trained models to current Google account

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Training and Evaluation

#### Setup ClearML

Go ahead and sign-up/sign-in to [AI Infrastructure Platform | Maximize AI Performance & Scalability | ClearML](https://clear.ml/)

After that, go to Settings -> Workspace -> Create new credentials

The new credentials will be created and shows two options:

Local Python (Recommended)
Jupyter Notebook
Both actually are the same things, it only differs on how to use the new credentials.

This time, use the clearml CLI app to consume the credentials, when prompted, paste it.

In [ ]:
!pip install clearml

In [ ]:
!clearml-init

In [ ]:
!pip install torch torchvision matplotlib

In [ ]:
#@title <b>Time Out Preventer (Advanced) </b></strong>
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
#@markdown **Run this code to prevent Google Colab from Timeout**
from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True)
if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button");
      btn.click()
      }

    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect");
      btn.click()
      }
    }

  setInterval(ClickConnect,60000)
  '''))

### Training Options (Choose one of these)

#### (1) Training with early stopping - Recommended

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
import time
import os
from clearml import Task, Logger, OutputModel


# 🔁 Transformasi
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# 📁 Load dataset
train_dataset = datasets.ImageFolder("dataset/train", transform=transform)
val_dataset = datasets.ImageFolder("dataset/test", transform=transform)
class_names = train_dataset.classes

print("Label mapping:", train_dataset.class_to_idx)
print("Train distribusi:", Counter([label for _, label in train_dataset]))
print("Val distribusi:", Counter([label for _, label in val_dataset]))

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# ⚙️ Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)
model.fc = nn.Linear(model.fc.in_features, len(class_names))
model = model.to(device)

# Inbalanced dataset
labels = [label for _, label in train_dataset]
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(labels),
                                     y=labels)

weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

# 🔁 Training loop
epochs = 32
patience = 12
train_accs, val_accs = [], []
train_losses, val_losses = [], []
best_val_acc = 0
early_stop_counter = 0

# Model Storage
model_name = model.__class__.__name__
best_model_path = f"/content/drive/MyDrive/{model_name}_best_model.pt"
latest_model_path = f"/content/drive/MyDrive/{model_name}_latest_model.pt"
os.makedirs(os.path.dirname("/content/drive/MyDrive/Cool Lee Yeah/8th Semester/Skripsi/AI Models"), exist_ok=True)

task = Task.init(
    project_name="EcoSort CNN",
    task_name=f"{model_name} Training {time.strftime('%a, %b %-d, %Y - %H:%M:%S')}",
    task_type=Task.TaskTypes.training
)
logger = task.get_logger()

# 🔍 Logging sample predictions
def log_predictions(images, labels, preds, class_names, epoch):
    fig, axs = plt.subplots(1, 5, figsize=(15, 3))
    for i in range(min(5, len(images))):
        img = images[i].cpu().permute(1, 2, 0) * 0.5 + 0.5  # unnormalize
        axs[i].imshow(img.numpy())
        axs[i].axis('off')
        axs[i].set_title(f"T: {class_names[labels[i]]}\nP: {class_names[preds[i]]}")
    logger.report_matplotlib_figure(title="Sample Predictions", series="Validation", figure=fig, iteration=epoch)
    plt.close(fig)

# 🏃 Training starts
try:
  for epoch in range(epochs):
      start_time = time.time()

      model.train()
      train_loss = 0
      correct, total = 0, 0

      for images, labels in train_loader:
          images, labels = images.to(device), labels.to(device)
          optimizer.zero_grad()
          outputs = model(images)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          train_loss += loss.item()
          _, preds = torch.max(outputs, 1)
          correct += torch.sum(preds == labels)
          total += labels.size(0)

      train_acc = correct / total
      train_accs.append(train_acc.item())
      train_losses.append(train_loss / len(train_loader))

      # ClearML Logging (Train)
      logger.report_scalar("Accuracy", "Train", value=train_acc.item(), iteration=epoch)
      logger.report_scalar("Loss", "Train", value=train_loss / len(train_loader), iteration=epoch)
      logger.report_scalar("LR", "Learning Rate", value=optimizer.param_groups[0]['lr'], iteration=epoch)

      # 🔍 Validasi
      model.eval()
      val_loss = 0
      correct, total = 0, 0
      y_true, y_pred = [], []
      last_batch_images, last_batch_labels, last_batch_preds = None, None, None

      with torch.no_grad():
          for images, labels in val_loader:
              images, labels = images.to(device), labels.to(device)
              outputs = model(images)
              loss = criterion(outputs, labels)

              val_loss += loss.item()
              _, preds = torch.max(outputs, 1)
              correct += torch.sum(preds == labels)
              total += labels.size(0)

              y_true.extend(labels.cpu().numpy())
              y_pred.extend(preds.cpu().numpy())

              last_batch_images = images
              last_batch_labels = labels
              last_batch_preds = preds

      val_acc = correct / total
      val_accs.append(val_acc.item())
      val_losses.append(val_loss / len(val_loader))

      # ClearML Logging (Val)
      logger.report_scalar("Accuracy", "Validation", value=val_acc.item(), iteration=epoch)
      logger.report_scalar("Loss", "Validation", value=val_loss / len(val_loader), iteration=epoch)

      # Classification Report
      report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)
      for class_name in class_names:
          logger.report_scalar("Precision", class_name, value=report[class_name]["precision"], iteration=epoch)
          logger.report_scalar("Recall", class_name, value=report[class_name]["recall"], iteration=epoch)
          logger.report_scalar("F1-Score", class_name, value=report[class_name]["f1-score"], iteration=epoch)
      logger.report_scalar("F1-Score", "Macro Avg", value=report["macro avg"]["f1-score"], iteration=epoch)

      # Confusion Matrix
      cm = confusion_matrix(y_true, y_pred)
      fig, ax = plt.subplots(figsize=(6, 6))
      disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
      disp.plot(cmap="Blues", ax=ax)
      plt.title("Confusion Matrix")
      logger.report_matplotlib_figure(title="Confusion Matrix", series="Validation", figure=fig, iteration=epoch)
      plt.close(fig)

      # Sample Prediction Logging
      log_predictions(last_batch_images, last_batch_labels, last_batch_preds, class_names, epoch)
      print(f"🔁 Epoch {epoch+1}/{epochs} - Train Acc: {train_acc:.4f} - Val Acc: {val_acc:.4f}")

      # Save model
      if val_acc >= best_val_acc:
          best_val_acc = val_acc
          early_stop_counter = 0
          torch.save(model.state_dict(), best_model_path)
          print(f"🏆 (Best) Model saved")

      else:
          torch.save(model.state_dict(), latest_model_path)
          print(f"📦 (Latest) Model saved")
          early_stop_counter += 1
          if early_stop_counter >= patience:
              print("⏹️ Early stopping triggered.")
              break

      duration = time.time() - start_time
      eta = (epochs - epoch - 1) * duration
      print(f"⏱️ Epoch time: Took {duration:.2f}s - ETA: ~{eta/60:.1f} min")
      logger.report_scalar("Epoch Time (sec)", "Duration", value=duration, iteration=epoch)
      print("\n")

  # 🎉 Done
  print("=== Final Classification Report ===")
  print(classification_report(y_true, y_pred, target_names=class_names))

  task.mark_completed()

except Exception:
    task.mark_failed()


#### (2) Training without early stopping

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter
import time

# Transform
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

# Dataset
train_dataset = datasets.ImageFolder("dataset/train", transform=transform)
class_names = train_dataset.classes
print("Label mapping:", train_dataset.class_to_idx)
print("Distribusi:", Counter([label for _, label in train_dataset]))

val_dataset = datasets.ImageFolder("dataset/test", transform=transform)
print("Label mapping:", val_dataset.class_to_idx)
print("Distribusi:", Counter([label for _, label in val_dataset]))

# Split
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)

# Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(class_names))
model = model.to(device)

# Loss & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training
epochs = 6
train_accs, val_accs = [], []
best_val_acc = 0

for epoch in range(epochs):
    model.train()
    correct, total, loss_total = 0, 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        loss_total += loss.item()
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels)
        total += labels.size(0)

    train_acc = correct / total
    train_accs.append(train_acc.item())
    print(f"Epoch {epoch+1}/{epochs} - Train Acc: {train_acc:.4f}")

    # Validasi
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels)
            total += labels.size(0)

    val_acc = correct / total
    val_accs.append(val_acc.item())
    print(f"            → Val Acc: {val_acc:.4f}")

# Simpan
torch.save(model.state_dict(), "model_cnn.pt")
print("✅ Model disimpan.")

### Evaluation

In [ ]:
# 📊 Evaluasi dengan laporan & Confusion Matrix
model.eval()
y_true, y_pred = [], []

for images, labels in val_loader:
    images = images.to(device)
    outputs = model(images)
    _, preds = torch.max(outputs, 1)

    y_true.extend(labels.numpy())
    y_pred.extend(preds.cpu().numpy())

print("\n=== Classification Report ===")
print(classification_report(y_true, y_pred, target_names=class_names))

# Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap="Blues")
plt.title("Confusion Matrix")
plt.show()

# Plot akurasi training & val
plt.plot(train_accs, label="Train")
plt.plot(val_accs, label="Validation")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.title("Train vs Validation Accuracy")
plt.legend()
plt.grid(True)
plt.show()